In [10]:
import pandas as pd
from tqdm.auto import tqdm

# Load the movies.csv file
movies_df = pd.read_csv("D:\movie_data\movies.csv", encoding='utf-8')

# Show the head of the dataframe
print(movies_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [11]:
# Print the shape of the movies dataframe
print(movies_df.shape)

(9742, 3)


In [14]:
# Load the ratings.csv file
ratings_df = pd.read_csv("D:/movie_data/ratings.csv")

# Show the head of the dataframe
print(ratings_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [15]:
# Print the shape of the ratings dataframe
print(ratings_df.shape)

(100836, 4)


In [16]:
# Calculate the number of unique userIds in the ratings dataframe
unique_user_ids = ratings_df['userId'].nunique()
print(unique_user_ids)

610


In [17]:
# Find the movie with the maximum number of user ratings
movie_rating_counts = ratings_df['movieId'].value_counts()
max_ratings_movie_id = movie_rating_counts.idxmax()
max_ratings_count = movie_rating_counts.max()

# Retrieve the movie title corresponding to the movie with the maximum number of ratings
max_ratings_movie_title = movies_df.loc[movies_df['movieId'] == max_ratings_movie_id, 'title'].values[0]

print(max_ratings_movie_id, max_ratings_movie_title, max_ratings_count)

356 Forrest Gump (1994) 329


In [18]:
# Find the movieId for 'Terminator 2: Judgment Day (1991)'
terminator_movie_id = movies_df[movies_df['title'] == 'Terminator 2: Judgment Day (1991)']['movieId'].iloc[0]

# Calculate the average rating for 'Terminator 2: Judgment Day (1991)'
terminator_avg_rating = ratings_df[ratings_df['movieId'] == terminator_movie_id]['rating'].mean()
print(terminator_avg_rating)

3.970982142857143


In [19]:
# Calculate the average rating for each movie and count the number of ratings
movie_ratings = ratings_df.groupby('movieId').agg({'rating': ['mean', 'count']})

# Flatten the multi-level columns
movie_ratings.columns = ['average_rating', 'rating_count']

# Filter out movies with a low number of ratings to get a more accurate picture of popularity
min_ratings_threshold = 100
popular_movies = movie_ratings[movie_ratings['rating_count'] > min_ratings_threshold]

# Find the movie with the highest average rating among those with more than the threshold of ratings
most_popular_movie_id = popular_movies['average_rating'].idxmax()
most_popular_movie = movies_df[movies_df['movieId'] == most_popular_movie_id]

# Display the title and average rating of the most popular movie
most_popular_movie_title = most_popular_movie['title'].iloc[0]
most_popular_movie_avg_rating = popular_movies.loc[most_popular_movie_id, 'average_rating']

print('Most Popular Movie Based on Average User Ratings:')
print('Title:', most_popular_movie_title)
print('Average Rating:', most_popular_movie_avg_rating)

Most Popular Movie Based on Average User Ratings:
Title: Shawshank Redemption, The (1994)
Average Rating: 4.429022082018927


In [23]:
# Find the top 5 popular movies based on the number of user ratings
top_5_popular_movies = popular_movies.sort_values('rating_count', ascending=False).head(5)
top_5_popular_movies

,average_rating,rating_count
movieId,,
356,4.164134,329
318,4.429022,317
296,4.197068,307
593,4.161290,279
2571,4.192446,278


In [24]:
# Merge the average ratings with the movies dataframe to get the movie titles for the top 5 popular movies
top_5_popular_movies_with_titles = top_5_popular_movies.merge(movies_df, on='movieId', how='inner')
top_5_popular_movies_with_titles[['title', 'average_rating', 'rating_count']]

,title,average_rating,rating_count
0,Forrest Gump (1994),4.164134,329
1,"Shawshank Redemption, The (1994)",4.429022,317
2,Pulp Fiction (1994),4.197068,307
3,"Silence of the Lambs, The (1991)",4.161290,279
4,"Matrix, The (1999)",4.192446,278


In [25]:
# Find the third most popular Sci-Fi movie based on the number of user ratings
sci_fi_movies = movies_df[movies_df['genres'].str.contains('Sci-Fi', case=False)]
sci_fi_movie_ratings = popular_movies.merge(sci_fi_movies, on='movieId', how='inner')
third_most_popular_sci_fi_movie = sci_fi_movie_ratings.sort_values('rating_count', ascending=False).iloc[2]
third_most_popular_sci_fi_movie

movieId                                        480
average_rating                                3.75
rating_count                                   238
title                         Jurassic Park (1993)
genres            Action|Adventure|Sci-Fi|Thriller
Name: 6, dtype: object

In [26]:
links_df=pd.read_csv("D:\movie_data\links.csv")

In [27]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [28]:
links_df[links_df['imdbId'] == links_df['imdbId'].max()]

,movieId,imdbId,tmdbId
9740,193587,8391976,483455.0


In [32]:
# Display the columns of the final merged dataframe to identify the column containing the IMDB ratings
final_merged_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'imdbId', 'tmdbId', 'title',
       'genres'],
      dtype='object')

In [34]:
"{\"df_name\":\"final_merged_df\",\"columns_names\":[\"genres\"],\"new_columns\":{\"imdb_rating\":\"The IMDB rating of the movie\"}}"

'{"df_name":"final_merged_df","columns_names":["genres"],"new_columns":{"imdb_rating":"The IMDB rating of the movie"}}'

In [36]:
# Find the movie with the highest IMDB rating among Sci-Fi movies
scifi_movies = final_merged_df[final_merged_df['genres'].str.contains('Sci-Fi')]
highest_imdb_rating_scifi_movie = scifi_movies[scifi_movies['imdbId'] == scifi_movies['imdbId'].max()]
highest_imdb_rating_scifi_movie[['movieId', 'title', 'imdbId']]

,movieId,title,imdbId
100442,179053,2048: Nowhere to Run (2017),7387408
